#    ETL PROJECT
### Spotify x Billboard Music Charts
#### Todd Tuchek, Lesly Sok and Raul Villa

In [73]:
# Dependencies
import pandas as pd
import sqlalchemy
import numpy as np
from sqlalchemy import create_engine
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, inspect
from sqlalchemy import Column, Integer, String, Float
from sqlalchemy.ext.declarative import declarative_base
Base = declarative_base()

## Extract
### Resources: https://www.kaggle.com/danield2255/data-on-songs-from-billboard-19992019 

In [2]:
# Import Spotify Data
spotify_data = "spotifyWeeklyTop200Streams_cleaned12.6.csv"

spotify_df = pd.read_csv(spotify_data)
spotify_df.head()

,Unnamed: 0,Name,Artist,Features,Streams,Week
0,0,In My Feelings,Drake,NaN,30747676,7/20/2018
1,1,Lucid Dreams,Juice WRLD,NaN,12930705,7/20/2018
2,2,Nonstop,Drake,NaN,12312859,7/20/2018
3,3,God is a woman,Ariana Grande,NaN,10771324,7/20/2018
4,4,SAD!,XXXTENTACION,NaN,10503061,7/20/2018


In [3]:
# Import the data from Billboard
billboard_data = "billboardHot100_1999-2019_cleaned_12.6.csv"

billboard_df = pd.read_csv(billboard_data)
billboard_df.head()

,Artists,Name,Weekly.rank,Peak.position,Weeks.on.chart,Week,Date,Genre,Writing.Credits,Lyrics,Features
0,17,NC,41,NaN,NaN,8/18/2018,8/3/2018,"Trap,Rap","Ryan meyer, Rich meyer, Johnny stevens, Allen ...","NC-17 \nOoh\nMe and my bitch, I swear we like ...",NaN
1,800,1,45,3.0,42.0,2/24/2018,4/28/2017,"Hip-Hop,Canada,DMV,Pop,Rap","Dylan wiggins, Andrew taggart, 6ix, Alessia ca...",1-800-273-8255 \nI've been on the low\nI been ...,NaN
2,800,1,40,3.0,41.0,2/17/2018,4/28/2017,"Hip-Hop,Canada,DMV,Pop,Rap","Dylan wiggins, Andrew taggart, 6ix, Alessia ca...",1-800-273-8255 \nI've been on the low\nI been ...,NaN
3,800,1,33,3.0,40.0,2/10/2018,4/28/2017,"Hip-Hop,Canada,DMV,Pop,Rap","Dylan wiggins, Andrew taggart, 6ix, Alessia ca...",1-800-273-8255 \nI've been on the low\nI been ...,NaN
4,800,1,46,3.0,39.0,2/3/2018,4/28/2017,"Hip-Hop,Canada,DMV,Pop,Rap","Dylan wiggins, Andrew taggart, 6ix, Alessia ca...",1-800-273-8255 \nI've been on the low\nI been ...,NaN


## Transform 

In [4]:
# Drop Features Column in Spotify DataFrame
spotify_df = spotify_df.drop(columns = ['Features', 'Unnamed: 0'])
spotify_df.head()

,Name,Artist,Streams,Week
0,In My Feelings,Drake,30747676,7/20/2018
1,Lucid Dreams,Juice WRLD,12930705,7/20/2018
2,Nonstop,Drake,12312859,7/20/2018
3,God is a woman,Ariana Grande,10771324,7/20/2018
4,SAD!,XXXTENTACION,10503061,7/20/2018


In [5]:
# Adjust the Spotify DataFrame so the start of each week lines up with Billboard
from datetime import timedelta
spotify_df["Week"] = pd.to_datetime(spotify_df["Week"])
spotify_df["one_day_ahead"] = spotify_df["Week"].apply(lambda x: x + timedelta(days=1))
spotify_df["one_day_ahead"] = spotify_df["one_day_ahead"].dt.strftime("%m/%d/%y")
spotify_df = spotify_df.drop(columns = ["Week"])
spotify_df.head()

,Name,Artist,Streams,one_day_ahead
0,In My Feelings,Drake,30747676,07/21/18
1,Lucid Dreams,Juice WRLD,12930705,07/21/18
2,Nonstop,Drake,12312859,07/21/18
3,God is a woman,Ariana Grande,10771324,07/21/18
4,SAD!,XXXTENTACION,10503061,07/21/18


In [6]:
# Rename "Columns"
spotify_df.rename(columns = {'Name': 'song_name',
                             'Artist': 'artist',
                             'Streams': 'weekly_streams',
                             'one_day_ahead': 'week_of'}, inplace = True)
spotify_df.head()

,song_name,artist,weekly_streams,week_of
0,In My Feelings,Drake,30747676,07/21/18
1,Lucid Dreams,Juice WRLD,12930705,07/21/18
2,Nonstop,Drake,12312859,07/21/18
3,God is a woman,Ariana Grande,10771324,07/21/18
4,SAD!,XXXTENTACION,10503061,07/21/18


In [7]:
billboard_df.head()

,Artists,Name,Weekly.rank,Peak.position,Weeks.on.chart,Week,Date,Genre,Writing.Credits,Lyrics,Features
0,17,NC,41,NaN,NaN,8/18/2018,8/3/2018,"Trap,Rap","Ryan meyer, Rich meyer, Johnny stevens, Allen ...","NC-17 \nOoh\nMe and my bitch, I swear we like ...",NaN
1,800,1,45,3.0,42.0,2/24/2018,4/28/2017,"Hip-Hop,Canada,DMV,Pop,Rap","Dylan wiggins, Andrew taggart, 6ix, Alessia ca...",1-800-273-8255 \nI've been on the low\nI been ...,NaN
2,800,1,40,3.0,41.0,2/17/2018,4/28/2017,"Hip-Hop,Canada,DMV,Pop,Rap","Dylan wiggins, Andrew taggart, 6ix, Alessia ca...",1-800-273-8255 \nI've been on the low\nI been ...,NaN
3,800,1,33,3.0,40.0,2/10/2018,4/28/2017,"Hip-Hop,Canada,DMV,Pop,Rap","Dylan wiggins, Andrew taggart, 6ix, Alessia ca...",1-800-273-8255 \nI've been on the low\nI been ...,NaN
4,800,1,46,3.0,39.0,2/3/2018,4/28/2017,"Hip-Hop,Canada,DMV,Pop,Rap","Dylan wiggins, Andrew taggart, 6ix, Alessia ca...",1-800-273-8255 \nI've been on the low\nI been ...,NaN


In [8]:
# Drop Columns on Billboard DataFrame
billboard_df = billboard_df.drop(columns = ['Peak.position', 
                                            'Weeks.on.chart', 
                                            'Date', 
                                            'Writing.Credits', 
                                            'Lyrics', 
                                            'Features'])
billboard_df.head()

,Artists,Name,Weekly.rank,Week,Genre
0,17,NC,41,8/18/2018,"Trap,Rap"
1,800,1,45,2/24/2018,"Hip-Hop,Canada,DMV,Pop,Rap"
2,800,1,40,2/17/2018,"Hip-Hop,Canada,DMV,Pop,Rap"
3,800,1,33,2/10/2018,"Hip-Hop,Canada,DMV,Pop,Rap"
4,800,1,46,2/3/2018,"Hip-Hop,Canada,DMV,Pop,Rap"


In [9]:
from datetime import timedelta
billboard_df["Week"] = pd.to_datetime(billboard_df["Week"])
billboard_df["Week"] = billboard_df["Week"].dt.strftime("%m/%d/%y")
billboard_df.head()

,Artists,Name,Weekly.rank,Week,Genre
0,17,NC,41,08/18/18,"Trap,Rap"
1,800,1,45,02/24/18,"Hip-Hop,Canada,DMV,Pop,Rap"
2,800,1,40,02/17/18,"Hip-Hop,Canada,DMV,Pop,Rap"
3,800,1,33,02/10/18,"Hip-Hop,Canada,DMV,Pop,Rap"
4,800,1,46,02/03/18,"Hip-Hop,Canada,DMV,Pop,Rap"


In [10]:
# split everything after (',') for Genre
from io import StringIO
billboard_df['Genre'] = billboard_df['Genre'].apply(lambda x: x.split(',')[0])
billboard_df

,Artists,Name,Weekly.rank,Week,Genre
0,17,NC,41,08/18/18,Trap
1,800,1,45,02/24/18,Hip-Hop
2,800,1,40,02/17/18,Hip-Hop
3,800,1,33,02/10/18,Hip-Hop
4,800,1,46,02/03/18,Hip-Hop
...,...,...,...,...,...
13277,"Zedd, Maren Morris, Grey",The Middle,11,03/10/18,Deutschland
13278,"Zedd, Maren Morris, Grey",The Middle,13,03/03/18,Deutschland
13279,"Zedd, Maren Morris, Grey",The Middle,17,02/24/18,Deutschland
13280,"Zedd, Maren Morris, Grey",The Middle,22,02/17/18,Deutschland


In [11]:
# Rename columns in Billboard DataFrame
billboard_df.rename(columns = {'Artists': 'artist',
                                'Name': 'song_name', 
                               'Weekly.rank': 'weekly_rank', 
                               'Week': 'week_of',
                               'Genre': 'genre'}, inplace = True )
billboard_df.head()

,artist,song_name,weekly_rank,week_of,genre
0,17,NC,41,08/18/18,Trap
1,800,1,45,02/24/18,Hip-Hop
2,800,1,40,02/17/18,Hip-Hop
3,800,1,33,02/10/18,Hip-Hop
4,800,1,46,02/03/18,Hip-Hop


In [12]:
# Strip all blank spaces for Artist and Song Names
billboard_df['artist'] = billboard_df['artist'].str.rstrip()
billboard_df['song_name'] = billboard_df['song_name'].str.rstrip()
billboard_df

,artist,song_name,weekly_rank,week_of,genre
0,17,NC,41,08/18/18,Trap
1,800,1,45,02/24/18,Hip-Hop
2,800,1,40,02/17/18,Hip-Hop
3,800,1,33,02/10/18,Hip-Hop
4,800,1,46,02/03/18,Hip-Hop
...,...,...,...,...,...
13277,"Zedd, Maren Morris, Grey",The Middle,11,03/10/18,Deutschland
13278,"Zedd, Maren Morris, Grey",The Middle,13,03/03/18,Deutschland
13279,"Zedd, Maren Morris, Grey",The Middle,17,02/24/18,Deutschland
13280,"Zedd, Maren Morris, Grey",The Middle,22,02/17/18,Deutschland


In [13]:
# Convert to Weekly Streams to string
spotify_df['weekly_streams'] = spotify_df['weekly_streams'].apply(str)
spotify_df['weekly_streams'].dtype

dtype('O')

####  Merging Billboard and Spotify DataFrame

In [14]:
# Merging Billboard CSV with Spotify with a left join
df_merged = pd.merge(billboard_df, spotify_df, how='left', on= ['song_name', 'week_of'])
df_merged.head(100)

,artist_x,song_name,weekly_rank,week_of,genre,artist_y,weekly_streams
0,17,NC,41,08/18/18,Trap,NaN,NaN
1,800,1,45,02/24/18,Hip-Hop,NaN,NaN
2,800,1,40,02/17/18,Hip-Hop,NaN,NaN
3,800,1,33,02/10/18,Hip-Hop,NaN,NaN
4,800,1,46,02/03/18,Hip-Hop,NaN,NaN
...,...,...,...,...,...,...,...
95,2 Chainz,Good Drank,92,02/11/17,Atlanta,2 Chainz,1440861
96,21 Savage,1.5,86,01/05/19,Motown,21 Savage,1848760
97,21 Savage,A Lot,49,06/08/19,East Coast,NaN,NaN
98,21 Savage,A Lot,46,06/01/19,East Coast,NaN,NaN


In [15]:
# Validating our Data
df_merged[df_merged["song_name"] == "4:00 AM"]

,artist_x,song_name,weekly_rank,week_of,genre,artist_y,weekly_streams
42,2 Chainz,4:00 AM,95,09/09/17,Atlanta,2 Chainz,1864342
43,2 Chainz,4:00 AM,90,09/02/17,Atlanta,2 Chainz,1997523
44,2 Chainz,4:00 AM,93,08/26/17,Atlanta,2 Chainz,2660105
45,2 Chainz,4:00 AM,83,08/19/17,Atlanta,2 Chainz,2865754
46,2 Chainz,4:00 AM,88,08/12/17,Atlanta,2 Chainz,2727864
47,2 Chainz,4:00 AM,83,08/05/17,Atlanta,2 Chainz,2866786
48,2 Chainz,4:00 AM,76,07/29/17,Atlanta,2 Chainz,3069167
49,2 Chainz,4:00 AM,65,07/22/17,Atlanta,2 Chainz,3393430
50,2 Chainz,4:00 AM,59,07/15/17,Atlanta,2 Chainz,3517378
51,2 Chainz,4:00 AM,55,07/08/17,Atlanta,2 Chainz,3595373


In [16]:
# Dropping Artist column from Spotify csv
df_merged = df_merged.drop(columns = ['artist_y'])
df_merged.head()

,artist_x,song_name,weekly_rank,week_of,genre,weekly_streams
0,17,NC,41,08/18/18,Trap,NaN
1,800,1,45,02/24/18,Hip-Hop,NaN
2,800,1,40,02/17/18,Hip-Hop,NaN
3,800,1,33,02/10/18,Hip-Hop,NaN
4,800,1,46,02/03/18,Hip-Hop,NaN


In [17]:
# Renaming Column name
df_merged.rename(columns = {'artist_x': 'artist'}, inplace=True)

In [18]:
df_merged[df_merged["song_name"] == "4:00 AM"]

,artist,song_name,weekly_rank,week_of,genre,weekly_streams
42,2 Chainz,4:00 AM,95,09/09/17,Atlanta,1864342
43,2 Chainz,4:00 AM,90,09/02/17,Atlanta,1997523
44,2 Chainz,4:00 AM,93,08/26/17,Atlanta,2660105
45,2 Chainz,4:00 AM,83,08/19/17,Atlanta,2865754
46,2 Chainz,4:00 AM,88,08/12/17,Atlanta,2727864
47,2 Chainz,4:00 AM,83,08/05/17,Atlanta,2866786
48,2 Chainz,4:00 AM,76,07/29/17,Atlanta,3069167
49,2 Chainz,4:00 AM,65,07/22/17,Atlanta,3393430
50,2 Chainz,4:00 AM,59,07/15/17,Atlanta,3517378
51,2 Chainz,4:00 AM,55,07/08/17,Atlanta,3595373


## Create DataBase connection


In [19]:
# SQL
from sqlalchemy import create_engine
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, inspect
from sqlalchemy import Column, Integer, String, Float, Date, Table 
from sqlalchemy.ext.declarative import declarative_base
Base = declarative_base()
import config

In [21]:
class Merged(Base):
  __tablename__ = "merged"
  artist = Column(String, primary_key=True)
  song_name = Column(String)
  weekly_rank = Column(Date)
  week_of = Column(Integer)
  genre = Column(String)
  weekly_streams = Column(Integer)

In [22]:
# Right now, this table only exists in python and not in the actual database
Base.metadata.tables

immutabledict({'merged': Table('merged', MetaData(bind=None), Column('artist', String(), table=<merged>, primary_key=True, nullable=False), Column('song_name', String(), table=<merged>), Column('weekly_rank', Date(), table=<merged>), Column('week_of', Integer(), table=<merged>), Column('genre', String(), table=<merged>), Column('weekly_streams', Integer(), table=<merged>), schema=None)})

In [23]:
# This is where we create our tables in the database
Base.metadata.create_all(engine)

In [24]:
engine.table_names()

['merged']

## Load DataFrames into database

In [25]:
df_merged.to_sql(name='merged', con=engine, if_exists='replace', index=False)

In [26]:
# Read the Merged Database in Pandas
pd.read_sql_query('''SELECT * FROM merged;''', engine)

,artist,song_name,weekly_rank,week_of,genre,weekly_streams
0,17,NC,41,08/18/18,Trap,None
1,800,1,45,02/24/18,Hip-Hop,None
2,800,1,40,02/17/18,Hip-Hop,None
3,800,1,33,02/10/18,Hip-Hop,None
4,800,1,46,02/03/18,Hip-Hop,None
...,...,...,...,...,...,...
13383,"Zedd, Maren Morris, Grey",The Middle,11,03/10/18,Deutschland,7664089
13384,"Zedd, Maren Morris, Grey",The Middle,13,03/03/18,Deutschland,7714406
13385,"Zedd, Maren Morris, Grey",The Middle,17,02/24/18,Deutschland,7006684
13386,"Zedd, Maren Morris, Grey",The Middle,22,02/17/18,Deutschland,6827589


In [27]:
df_merged = df_merged.rename(columns={'artist': 'Artist','song_name': 'Song Name',
                                      'weekly_rank': 'Weekly Rank',
                                      'week_of': 'Week Of', 'genre': 'Genre',
                                      'weekly_streams': 'Weekly Streams'})
df_merged.head()

,Artist,Song Name,Weekly Rank,Week Of,Genre,Weekly Streams
0,17,NC,41,08/18/18,Trap,NaN
1,800,1,45,02/24/18,Hip-Hop,NaN
2,800,1,40,02/17/18,Hip-Hop,NaN
3,800,1,33,02/10/18,Hip-Hop,NaN
4,800,1,46,02/03/18,Hip-Hop,NaN


In [28]:
# Take a peak at the last few rows of Dataframe for consistency
df_merged.tail()

,Artist,Song Name,Weekly Rank,Week Of,Genre,Weekly Streams
13383,"Zedd, Maren Morris, Grey",The Middle,11,03/10/18,Deutschland,7664089
13384,"Zedd, Maren Morris, Grey",The Middle,13,03/03/18,Deutschland,7714406
13385,"Zedd, Maren Morris, Grey",The Middle,17,02/24/18,Deutschland,7006684
13386,"Zedd, Maren Morris, Grey",The Middle,22,02/17/18,Deutschland,6827589
13387,"Zedd, Maren Morris, Grey",The Middle,23,02/10/18,Deutschland,6689374


In [29]:
# Create the inspector and connect it to the engine
inspector = inspect(engine)

In [30]:
# Using the inspector to print the column names within the 'merged' table and its types
columns = inspector.get_columns('merged')
for column in columns:
    print(column["name"], column["type"])

artist TEXT
song_name TEXT
weekly_rank BIGINT
week_of TEXT
genre TEXT
weekly_streams TEXT


In [31]:
df_merged.dtypes

Artist            object
Song Name         object
Weekly Rank        int64
Week Of           object
Genre             object
Weekly Streams    object
dtype: object

In [37]:
#df_merged['Weekly Streams'].fillna(0)
df_merged = df_merged.fillna(0)
df_merged

,Artist,Song Name,Weekly Rank,Week Of,Genre,Weekly Streams
0,17,NC,41,08/18/18,Trap,0
1,800,1,45,02/24/18,Hip-Hop,0
2,800,1,40,02/17/18,Hip-Hop,0
3,800,1,33,02/10/18,Hip-Hop,0
4,800,1,46,02/03/18,Hip-Hop,0
...,...,...,...,...,...,...
13383,"Zedd, Maren Morris, Grey",The Middle,11,03/10/18,Deutschland,7664089
13384,"Zedd, Maren Morris, Grey",The Middle,13,03/03/18,Deutschland,7714406
13385,"Zedd, Maren Morris, Grey",The Middle,17,02/24/18,Deutschland,7006684
13386,"Zedd, Maren Morris, Grey",The Middle,22,02/17/18,Deutschland,6827589


In [38]:
# Using the inspector to print the column names within the 'merged' table and its types
df_merged['Weekly Streams'] = df_merged['Weekly Streams'].apply(int)
df_merged['Weekly Streams'].dtype

dtype('int64')

In [49]:
# find counts unique values in a column
artist_counts = df_merged["Artist"].value_counts()
artist_counts

Post Malone       319
Drake             314
Cardi B           223
Ariana Grande     207
XXXTENTACION      196
                 ... 
Chris Blue          1
Bradley Cooper      1
Jennifer Lopez      1
Janelle Monae       1
Jason Mraz          1
Name: Artist, Length: 476, dtype: int64

In [71]:
# find count of how many times your favorite artist was in the top 100
favorite_artist = session.query(Merged).filter_by(artist='2 Chainz').count()
print(favorite_artist)

54


In [72]:
# find the number of times your favorite song was in the top 100
session = Session(bind=engine)
favorite_song = session.query(Merged).filter_by(song_name='Zombie').count()
print(favorite_song)

18


In [67]:
highest_streams = df_merged.sort_values(by="Weekly Streams", ascending=False)
highest_streams.head(25)

,Artist,Song Name,Weekly Rank,Week Of,Genre,Weekly Streams
3772,Drake,In My Feelings,1,07/21/18,Bounce,30747676
3722,Drake,God's Plan,1,02/03/18,Hip-Hop,28708450
3773,Drake,In My Feelings,6,07/14/18,Bounce,24879464
3721,Drake,God's Plan,1,02/10/18,Hip-Hop,24209854
3719,Drake,God's Plan,1,02/24/18,Hip-Hop,23813466
3771,Drake,In My Feelings,1,07/28/18,Bounce,23616199
3718,Drake,God's Plan,1,03/03/18,Hip-Hop,23327237
3720,Drake,God's Plan,1,02/17/18,Hip-Hop,22071986
3717,Drake,God's Plan,1,03/10/18,Hip-Hop,20829362
3819,Drake,Nice For What,1,04/21/18,Bounce,19830250


In [51]:
# find counts unique values in a column
song_counts = df_merged["Song Name"].value_counts()
song_counts

Unforgettable           104
Despacito                77
Shape Of You             58
Perfect                  56
Better Now               52
                       ... 
Business Is Business      1
Black SpiderMan           1
Listen Before I Go        1
Took His Time             1
Let It All Work Out       1
Name: Song Name, Length: 1369, dtype: int64

In [48]:
# find count of how many times your favorite artist was in the top 100
weekly_steams = df_merged.groupby('Artist').agg({'Weekly Streams': ['count']})
print(weekly_steams)

                    Weekly Streams
                             count
Artist                            
17                               1
2 Chainz                        54
21 Savage                      110
5 Seconds Of Summer             54
6LACK                            9
...                            ...
blackbear                       25
gnash                            1
lovelytheband                   22
migos                           25
twenty one pilots               29

[476 rows x 1 columns]


In [70]:
lowest_streams = df_merged["Artist"].min()
highes_streams = df_merged["Weekly Streams"].max()
print(highes_streams)

30747676
